*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-07-19 13:36:50--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  19.8MB/s    in 1.3s    

2021-07-19 13:36:52 (19.8 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
df = pd.merge(df_books, df_ratings, on='isbn')
print(df.head())

         isbn                title                author   user  rating
0  0195153448  Classical Mythology    Mark P. O. Morford      2     0.0
1  0002005018         Clara Callan  Richard Bruce Wright      8     5.0
2  0002005018         Clara Callan  Richard Bruce Wright  11400     0.0
3  0002005018         Clara Callan  Richard Bruce Wright  11676     8.0
4  0002005018         Clara Callan  Richard Bruce Wright  41385     0.0


In [14]:
vc = df['user'].value_counts()
df = df[df['user'].isin(vc[vc>200].index)]

vc = df['rating'].value_counts()
df = df[df['rating'].isin(vc[vc>100].index)]

print(df.head())

          isbn  ... rating
3   0002005018  ...    8.0
6   0002005018  ...    0.0
7   0002005018  ...    0.0
10  0002005018  ...    0.0
21  0374157065  ...   10.0

[5 rows x 5 columns]


In [17]:
df = df.drop_duplicates(['title', 'user'])

In [18]:
df_pivot = df.pivot_table(index = 'title', columns='user', values='rating').fillna(0)
print(df_pivot.head())

user                                                254     ...  278418
title                                                       ...        
 A Light in the Storm: The Civil War Diary of A...     0.0  ...     0.0
 Always Have Popsicles                                 0.0  ...     0.0
 Apple Magic (The Collector's series)                  0.0  ...     0.0
 Beyond IBM: Leadership Marketing and Finance f...     0.0  ...     0.0
 Clifford Visita El Hospital (Clifford El Gran ...     0.0  ...     0.0

[5 rows x 811 columns]


In [19]:
df_matrix = csr_matrix(df_pivot.values)

In [20]:
model = NearestNeighbors(metric='cosine', n_neighbors=5)
model.fit(df_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [32]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Get index
    x = df_pivot.index.get_loc(book)
    #use model nearest neighbors.
    distances, indices = model.kneighbors(df_pivot.iloc[x,:].values.reshape(1, -1), n_neighbors= 8)
    
    recommended_books = []
    for x in range(1,6):
        bookrecommended = [df_pivot.index[indices.flatten()[x]], distances.flatten()[x]]
        recommended_books.append(bookrecommended)
    recommended_books = [book, recommended_books]
    return recommended_books



In [38]:
books = get_recommends("Always Have Popsicles")
print(books)

KeyError: ignored

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [36]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Blue Diary', 0.7066704], ['The Lovely Bones: A Novel', 0.72582525], ['Dry: A Memoir', 0.7407396], ['I Know This Much Is True', 0.7416087], ['Errands', 0.74315697]]]
You haven't passed yet. Keep trying!
